In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import time
sns.set()

In [2]:
def get_vocab(file, lower = False):
    with open(file, 'r') as fopen:
        data = fopen.read()
    if lower:
        data = data.lower()
    vocab = list(set(data))
    return data, vocab

def embed_to_onehot(data, vocab):
    onehot = np.zeros((len(data), len(vocab)), dtype = np.float32)
    for i in range(len(data)):
        onehot[i, vocab.index(data[i])] = 1.0
    return onehot

In [3]:
text, text_vocab = get_vocab('consumer.h', lower = False)
onehot = embed_to_onehot(text, text_vocab)

In [4]:
learning_rate = 0.01
batch_size = 64
sequence_length = 12
epoch = 1000
num_layers = 2
size_layer = 128
possible_batch_id = range(len(text) - sequence_length - 1)
dimension = onehot.shape[1]
epsilon = 1e-8

U = np.random.randn(size_layer, dimension) / np.sqrt(size_layer)
U_g = np.zeros(U.shape)
W = np.random.randn(size_layer, size_layer) / np.sqrt(size_layer)
W_g = np.zeros(W.shape)
V = np.random.randn(dimension, size_layer) / np.sqrt(dimension)
V_g = np.zeros(V.shape)

In [5]:
def tanh(x, grad=False):
    if grad:
        output = np.tanh(x)
        return (1.0 - np.square(output))
    else:
        return np.tanh(x)
    
def softmax(x):
    exp_scores = np.exp(x - np.max(x))
    return exp_scores / (np.sum(exp_scores, axis=1, keepdims=True) + 1e-8)

def derivative_softmax_cross_entropy(x, y):
    delta = softmax(x)
    delta[range(X.shape[0]), y] -= 1
    return delta

def forward_multiply_gate(w, x):
    return np.dot(w, x)

def backward_multiply_gate(w, x, dz):
    dW = np.dot(dz.T, x)
    dx = np.dot(w.T, dz.T)
    return dW, dx

def forward_add_gate(x1, x2):
    return x1 + x2

def backward_add_gate(x1, x2, dz):
    dx1 = dz * np.ones_like(x1)
    dx2 = dz * np.ones_like(x2)
    return dx1, dx2

def cross_entropy(Y_hat, Y, epsilon=1e-12):
    Y_hat = np.clip(Y_hat, epsilon, 1. - epsilon)
    N = Y_hat.shape[0]
    return -np.sum(np.sum(Y * np.log(Y_hat+1e-9))) / N

def forward_recurrent(x, prev_state, U, W, V):
    mul_u = forward_multiply_gate(x, U.T)
    mul_w = forward_multiply_gate(prev_state, W.T)
    add_previous_now = forward_add_gate(mul_u, mul_w)
    current_state = tanh(add_previous_now)
    mul_v = forward_multiply_gate(current_state, V.T)
    return (mul_u, mul_w, add_previous_now, current_state, mul_v)

def backward_recurrent(x, prev_state, U, W, V, d_mul_v, saved_graph):
    mul_u, mul_w, add_previous_now, current_state, mul_v = saved_graph
    dV, dcurrent_state = backward_multiply_gate(V, current_state, d_mul_v)
    dadd_previous_now = tanh(add_previous_now, True) * dcurrent_state.T
    dmul_w, dmul_u = backward_add_gate(mul_w, mul_u, dadd_previous_now)
    dW, dprev_state = backward_multiply_gate(W, prev_state, dmul_w)
    dU, dx = backward_multiply_gate(U, x, dmul_u)
    return (dprev_state, dU, dW, dV)

In [6]:
for i in range(epoch):
    batch_x = np.zeros((batch_size, sequence_length, dimension))
    batch_y = np.zeros((batch_size, sequence_length, dimension))
    batch_id = random.sample(possible_batch_id, batch_size)
    prev_s = np.zeros((batch_size, size_layer))
    for n in range(sequence_length):
        id1 = [k + n for k in batch_id]
        id2 = [k + n + 1 for k in batch_id]
        batch_x[:,n,:] = onehot[id1, :]
        batch_y[:,n,:] = onehot[id2, :]
    layers = []
    out_logits = np.zeros((batch_size, sequence_length, dimension))
    for n in range(sequence_length):
        layers.append(forward_recurrent(batch_x[:,n,:], prev_s, U, W, V))
        prev_s = layers[-1][3]
        out_logits[:, n, :] = layers[-1][-1]
    probs = softmax(out_logits.reshape((-1, dimension)))
    y = np.argmax(batch_y.reshape((-1, dimension)),axis=1)
    accuracy = np.mean(np.argmax(probs,axis=1) == y)
    loss = cross_entropy(probs, batch_y.reshape((-1, dimension)))
    delta = probs
    delta[range(y.shape[0]), y] -= 1
    delta = delta.reshape((batch_size, sequence_length, dimension))
    dU = np.zeros(U.shape)
    dV = np.zeros(V.shape)
    dW = np.zeros(W.shape)
    prev_state = np.zeros((batch_size, size_layer))
    for n in range(sequence_length):
        d_mul_v = delta[:, n, :]
        dprev_s, dU_t, dW_t, dV_t = backward_recurrent(batch_x[:,n,:], prev_state, U, W, V, d_mul_v, layers[n])
        prev_state = layers[n][3]
        dV += dV_t
        dU += dU_t
        dW += dW_t
    U_g += dU ** 2
    U += -learning_rate * dU / np.sqrt(U_g + epsilon)
    V_g += dV ** 2
    V += -learning_rate * dV / np.sqrt(V_g + epsilon)
    W_g += dW ** 2
    W += -learning_rate * dW / np.sqrt(W_g + epsilon)
    if (i+1) % 50 == 0:
        print('epoch %d, loss %f, accuracy %f'%(i+1, loss, accuracy))

epoch 50, loss 2.192045, accuracy 0.490885
epoch 100, loss 1.801595, accuracy 0.571615
epoch 150, loss 1.777153, accuracy 0.559896
epoch 200, loss 1.680624, accuracy 0.583333
epoch 250, loss 1.381813, accuracy 0.658854
epoch 300, loss 1.309808, accuracy 0.673177
epoch 350, loss 1.257366, accuracy 0.684896
epoch 400, loss 1.152840, accuracy 0.718750
epoch 450, loss 1.214343, accuracy 0.675781
epoch 500, loss 1.187885, accuracy 0.690104
epoch 550, loss 1.129081, accuracy 0.725260
epoch 600, loss 1.072351, accuracy 0.718750
epoch 650, loss 1.180679, accuracy 0.701823
epoch 700, loss 1.060361, accuracy 0.743490
epoch 750, loss 1.124037, accuracy 0.720052
epoch 800, loss 1.187253, accuracy 0.712240
epoch 850, loss 1.157658, accuracy 0.717448
epoch 900, loss 0.906449, accuracy 0.772135
epoch 950, loss 1.138607, accuracy 0.714844
epoch 1000, loss 0.915181, accuracy 0.751302
